In [1]:
#Are Hs ever before end of pdb?
import glob
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import AllChem, GetPeriodicTable, RemoveHs
from rdkit.Chem import Descriptors
from rdkit.Chem import rdDistGeom
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
from scipy import stats

In [6]:
#Copied from https://github.com/gcorso/DiffDock/blob/main/datasets/process_mols.py
def read_molecule(molecule_file, sanitize=False, calc_charges=False, remove_hs=False):
    if molecule_file.endswith('.mol2'):
        mol = Chem.MolFromMol2File(molecule_file, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(molecule_file, sanitize=False, removeHs=False)
        mol = supplier[0]
    elif molecule_file.endswith('.pdbqt'):
        with open(molecule_file) as file:
            pdbqt_data = file.readlines()
        pdb_block = ''
        for line in pdbqt_data:
            pdb_block += '{}\n'.format(line[:66])
        mol = Chem.MolFromPDBBlock(pdb_block, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(molecule_file, sanitize=False, removeHs=False)
    else:
        raise ValueError('Expect the format of the molecule_file to be '
                         'one of .mol2, .sdf, .pdbqt and .pdb, got {}'.format(molecule_file))

    try:
        if sanitize or calc_charges:
            Chem.SanitizeMol(mol)

        if calc_charges:
            # Compute Gasteiger charges on the molecule.
            try:
                AllChem.ComputeGasteigerCharges(mol)
            except:
                warnings.warn('Unable to compute charges for the molecule.')

        if remove_hs:
            mol = Chem.RemoveHs(mol, sanitize=sanitize)
    except Exception as e:
        print(e)
        print("RDKit was unable to read the molecule.")
        return None

    return mol

In [7]:
#This has processed PDBbind data
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed/"

heavy_atom_after_h_list = []

#Check each complex
for ip, pdb_dir_name in enumerate(glob.glob(f"{pdb_dir}/*")):
    #print(pdb_dir_name)
    pdb_name = pdb_dir_name.split("/")[-1]
    
    #Progress track
    if ip % 10 == 0:
        print(f"On {pdb_name} index {ip}")
    #print(f"+++++++\n{pdb_name}\n+++++++")
    #Copied below from https://github.com/gcorso/DiffDock/blob/main/datasets/pdbbind.py
    for file in os.listdir(os.path.join(pdb_dir, pdb_name)):
        if file.endswith(".sdf") and 'rdkit' not in file:
            
            #list_pdb_names.append(pdb_name)
            
            lig = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=False, sanitize=True)
            #Pull w/o H for rmsd comparison w/HF output
            lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=True, sanitize=True)
            if lig is None and os.path.exists(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2")):  # read mol2 file if sdf file cannot be sanitized
                print('Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.')
                lig = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=False, sanitize=True)
                lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=True, sanitize=True)
            
                #Record atom info in each list
            #Get ID ref https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html, https://www.rdkit.org/docs/cppapi/classRDKit_1_1Atom.html
            #Could also explore https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html#rdkit.Chem.rdchem.Atom.GetProp
            found_H = False
            for a in lig.GetAtoms():
                anum = a.GetAtomicNum()
                if anum == 1:
                    found_H = True
                if found_H and anum != 1:
                    print(f"{pdb_name} has heavy atom after 1st H")
                    heavy_atom_after_h_list.append(pdb_name)

On 6ugp index 0
Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 31 32 33 34
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 31 32 33 34
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 4y7r index 10
Can't kekulize mol.  Unkekulized atoms: 60 61 62 63 64
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 60 61 62 63 64
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 55 56 57 58 59
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 55 56 57 58 59
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 16 17 18 23 24
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkeku

[19:12:32] Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 31 32 33 34
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 31 32 33 34
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 60 61 62 63 64
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 60 61 62 63 64
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 55 56 57 58 59
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 55 56 57 58 59
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 16 17 18 23 24
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 16 17 18 23 24
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5 6 8 9 10
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 0 1 3 4 5 6 8 9 10
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 33 34 35 37 3

On 5wlg index 90
On 4kiq index 100
Can't kekulize mol.  Unkekulized atoms: 63 64 65 66 67 68 69 70 71
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 63 64 65 66 67 68 69 70 71
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 5yjm index 110
Can't kekulize mol.  Unkekulized atoms: 67 68 69 70 71
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 67 68 69 70 71
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 2ykj index 120
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 12 13 14 15
RDKit was un

[19:12:32] Can't kekulize mol.  Unkekulized atoms: 63 64 65 66 67 68 69 70 71
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 63 64 65 66 67 68 69 70 71
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 67 68 69 70 71
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 67 68 69 70 71
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 8 10
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 12 13 14 15
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 12 13 14 15
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 10 11 16 17 18
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 10 11 16 17 18
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 13 20 21 22 26 27
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 13 20 21 22 26 27
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 5 6 24 25 26 27 28 29 40
[19:12:32] Can't kekulize mol.  Unkekulized atoms: 5 6 24 

Can't kekulize mol.  Unkekulized atoms: 38 39 40 41 42 43 44 45 46
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 38 39 40 41 42 43 44 45 46
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 3qpo index 190
On 3sut index 200
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8 10 11 12 13
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8 10 11 12 13
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 16 17 18 19 20 23 24
RDKit was unable to read the molecule

...

[19:13:26] Can't kekulize mol.  Unkekulized atoms: 9 11 12 15 21 22 23
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 9 11 12 15 21 22 23
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11 12 13 14 15
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11 12 13 14 15
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 6 7 8 9
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 6 7 8 9
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 13 14 15 17 18
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 13 14 15 17 18
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9
[19:13:26] Explicit valence for atom # 0 C greater than permitted
[19:13:26] Explicit valence for atom # 0 C greater than permitted
[19:13:26] Explicit valence for atom # 1 C, 6, is greater than permitted
[19:13:26] Explicit valence for atom # 1 C, 6, is greater than permitted
[19:13:26] Explicit va

Explicit valence for atom # 1 B, 4, is greater than permitted
RDKit was unable to read the molecule.
Explicit valence for atom # 1 B, 4, is greater than permitted
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 6etg index 18930
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 3oe9 index 18940
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9
RDKit

[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 1 2 18 19
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 1 2 18 19
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 1 8 9 15 24
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 1 8 9 15 24
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 7 11 12 13
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 7 11 12 13
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 5 6 19 20 21
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 5 6 19 20 21
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 75 76 77 78 79 80 81 82 83
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 75 76 77 78 79 80 81 82 83
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 27 28 29 30 31
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 27 28 29 30 31
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 19 20
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 19 20
[19:13:26] Can't kekulize mol.  

On 4wyp index 19020
Can't kekulize mol.  Unkekulized atoms: 4 5 6 9 10 11 18 19 20
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 4 5 6 9 10 11 18 19 20
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 6hmy index 19030
Can't kekulize mol.  Unkekulized atoms: 19 20 22 23 24
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 19 20 22 23 24
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 1 2 3 12 13 14 15
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 1 2 3 12 13 14 15
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 4qp1 index 19040
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 8 9
RDKit was unable to read the molecule.

[19:13:26] Can't kekulize mol.  Unkekulized atoms: 4 5 23 25 26
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 4 5 23 25 26
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 8 14 15 16
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 8 14 15 16
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 41 42 43 44 45
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 41 42 43 44 45
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 5 10 14 15 18 19 24 28 49
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 5 10 14 15 18 19 24 28 49
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24
[19:13:26] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9

In [9]:
sorted(list(set(heavy_atom_after_h_list)))

['1abt',
 '1ad8',
 '1bdl',
 '1fgi',
 '1gi6',
 '1gi8',
 '1gj4',
 '1gj5',
 '1idg',
 '1jbd',
 '1l2z',
 '1lxh',
 '1mwn',
 '1n7t',
 '1rgj',
 '1rst',
 '1sle',
 '1vwl',
 '1z2b',
 '1zkk',
 '2ez5',
 '2llq',
 '2mkr',
 '2qqs',
 '2r3c',
 '2r5b',
 '2r5d',
 '2xxn',
 '2y07',
 '3isw',
 '3k48',
 '3nij',
 '3o1d',
 '3o1e',
 '3pdh',
 '3pkn',
 '3pqz',
 '3puj',
 '3puk',
 '3qzt',
 '4dma',
 '4dow',
 '4edu',
 '4ejf',
 '4erz',
 '4esg',
 '4h36',
 '4hcz',
 '4htp',
 '4is6',
 '4ktu',
 '4mzj',
 '4mzk',
 '4mzl',
 '4n3w',
 '4nmq',
 '4nuf',
 '4o3t',
 '4o3u',
 '4ov5',
 '4pn1',
 '4pz8',
 '4r1e',
 '4wht',
 '4why']

In [11]:
#########
#Obtain list from a file
#Input:
#(a) f_name file name
#Output:
#(a) list_from_file list from the file
#########
def obtain_list_from_file(f_name):
    list_from_file = []
    file_for_list = open(f_name, "r")
    for entry_add in file_for_list:
        list_from_file.append(entry_add.rstrip())
    file_for_list.close()
    return list_from_file

In [12]:
test_list = obtain_list_from_file("/Users/dsharon/Documents/MIT/6.8701/Project/Data/For_ICLR_2024_02_05/PDB_Lists/test_pdb_ids.txt")
print(test_list)
print(len(test_list))

['6o3x', '6oiq', '6qr2', '6pya', '6qrg', '6e4c', '6i63', '6r7d', '6dql', '6jmf', '6oxu', '6qlr', '6qmr', '6quv', '6qw8', '6gbw', '6qfe', '6jse', '5zr3', '6rnu', '6qqv', '6rpg', '6oir', '6st3', '6g5u', '6i76', '6oio', '6ggf', '6nrj', '6o9c', '6qr7', '6mo2', '6p8y', '6qr3', '6e5s', '6pz4', '6cf7', '6i65', '6pgp', '6cjp', '6oxw', '6qto', '6gge', '5zk7', '6i78', '6nw3', '6s55', '6jsn', '6g2c', '6e3z', '6cyg', '6ueg', '6ufo', '6e3n', '6nrf', '6nt2', '6jsg', '6pgo', '6ufn', '6s56', '6g2f', '6g2e', '6hor', '6os6', '6npi', '6bqd', '6gj5', '6i75', '6os5', '6oxy', '6uhu', '6d08', '6n8x', '6qlp', '6e6v', '6k3l', '6e3p', '6dz0', '6g2b', '6qlo', '6e3m', '6g27', '6gzy', '6g24', '6r4k', '6e13', '6gj7', '6pno', '6o0h', '6oxp', '6qqq', '6uim', '6m7h', '6e3o', '6qtq', '6iby', '6i64', '6qr0', '6mhd', '6h14', '6pka', '6py0', '6qrc', '6pnm', '6qr1', '6oip', '6qr9', '6t6a', '6o3y', '6jsf', '6c85', '6ggb', '6nsv', '6nrg', '6i74', '6n4b', '6nd3', '6i8t', '6jt3', '6ibx', '6hzd', '6s9x', '6e7m', '6g3c', '6jsz',

In [13]:
print([t for t in test_list if t in heavy_atom_after_h_list])

[]


In [ ]:
#so no test set entries have the ordering issue

In [ ]:
#How many entries have issue with Hs in include_hs=False?

In [27]:
#This has processed PDBbind data
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed/"
inf_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Data/From_Hannes/HarmonicFlow/inference_output_last_xt"
h_when_remove_list = []
hs_removed_diff = []

#Check each complex
for ip, pdb_dir_name in enumerate(test_list):
    #print(pdb_dir_name)
    pdb_name = pdb_dir_name.split("/")[-1]
    
    #Progress track
    if ip % 10 == 0:
        print(f"On {pdb_name} index {ip}")
    #print(f"+++++++\n{pdb_name}\n+++++++")
    #Copied below from https://github.com/gcorso/DiffDock/blob/main/datasets/pdbbind.py
    for file in os.listdir(os.path.join(pdb_dir, pdb_name)):
        if file.endswith(".sdf") and 'rdkit' not in file:
            
            #list_pdb_names.append(pdb_name)
            
            lig = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=False, sanitize=True)
            #Pull w/o H for rmsd comparison w/HF output
            lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=True, sanitize=True)
            if lig is None and os.path.exists(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2")):  # read mol2 file if sdf file cannot be sanitized
                print('Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.')
                lig = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=False, sanitize=True)
                lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=True, sanitize=True)
            
            lighf = read_molecule(f"{inf_dir}/{pdb_name}_x20.pdb", remove_hs=True, sanitize=False)

            #Confirm atom count same, sometimes Hs get included
            count_l_atoms = len(lig_no_h.GetAtoms())
            count_lhf_atoms = len(lighf.GetAtoms())

            #If Hs are included only use non-H atom coordinates
            if count_l_atoms != count_lhf_atoms:
                h_when_remove_list.append(pdb_name)
                hs_removed_diff.append(count_l_atoms - count_lhf_atoms)
                
                #Try extra H removal step
                #Ref https://github.com/rdkit/rdkit/issues/2422
                lig_no_h_extra_removal = Chem.RemoveHs(lig_no_h)
                print(f"{len(lig_no_h_extra_removal.GetAtoms())} {len(lig_no_h.GetAtoms())}")

On 6o3x index 0
Can't kekulize mol.  Unkekulized atoms: 5 19 20 22 23
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 5 19 20 22 23
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 1 18 19 20
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 0 1 18 19 20
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17

[19:40:53] Can't kekulize mol.  Unkekulized atoms: 5 19 20 22 23
[19:40:53] Can't kekulize mol.  Unkekulized atoms: 5 19 20 22 23
[19:40:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
[19:40:53] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 18
[19:40:53] Can't kekulize mol.  Unkekulized atoms: 0 1 18 19 20
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 18 19 20
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 19 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 19 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 3 4 20 21 22
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 3 4 20 21 22
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 21 22 23 24 25 26 27 28 29
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 21 22 23 24 25 26 27 28 29
[19:40:54] Can't kekulize mol.  Un

Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 28
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 28
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 0 3 4 5 6 7 8 13 14
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 0 3 4 5 6 7 8 13 14
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
34 34
On 6c85 index 110
Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 22 23 24
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 22 23 24
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekul

[19:40:54] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 113 114 115 116 117
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 113 114 115 116 117
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 18 19 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 18 19 20 21
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 2 13 14
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 1 2 13 14
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 19 23
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 17 18 19 23
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6 7 8 9 17
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6 7 8 9 17
[19:

Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Explicit valence for atom # 0 C greater than permitted
RDKit was unable to read the molecule.
Explicit valence for atom # 0 C greater than permitted
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
On 6dz3 index 240
Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 12 13 14
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 12 13 14
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 4 5 9 10 11 12 13 14 19
RDKit was unable to read the molecule.
Can't kekulize mol.  Unkekulized atoms: 4 5 9 10 11 12 13 14 19
RDKit was unable to read the molecule.
Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.
Can't kekulize mol.  Unkekulized atoms: 8 

[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 5 6 7 9 10 13 18 19
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 0 5 6 7 9 10 13 18 19
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 19 20 21 22 23
[19:40:54] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 19 20 21 22 23


In [22]:
len(h_when_remove_list)

24

In [21]:
hs_removed_diff

[1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 3, 1, 1, 3, 4, 3, 1, 1, 1]